In [1]:
from typing import Any, Dict, List, Optional, Union, Tuple

import requests
import torch
from lm_polygraph import estimate_uncertainty
from lm_polygraph.estimators import *
from lm_polygraph.model_adapters.visual_whitebox_model import VisualWhiteboxModel
from PIL import Image

%load_ext autoreload
%autoreload 2
from transformers import (
    AutoModelForCausalLM,
    AutoModelForVision2Seq,
    AutoProcessor,
    AutoTokenizer,
)
from dataclasses import dataclass
import numpy as np

/home/jovyan/.mlspace/envs/focus_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

# Create whitebox model with image
url = "https://huggingface.co/microsoft/kosmos-2-patch14-224/resolve/main/snowman.png"
model = VisualWhiteboxModel(base_model, processor, image_urls=[url])

# Test with input text
input_text = ["<grounding>An image of"]

estimator = MaximumTokenProbability()
uncertainty = estimate_uncertainty(model, estimator, input_text=input_text)
print("Uncertainty estimation:", uncertainty)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`generation_config` default values have been modified to match model-specific defaults: {'no_repeat_ngram_size': 3, 'pad_token_id': 1, 'bos_token_id': 0, 'eos_token_id': 2}. If this is not desired, please set these values explicitly.


Uncertainty estimation: UncertaintyOutput(uncertainty=array([-0.43950686, -0.5575359 , -0.7364033 , -0.6337291 , -1.        ,
       -0.96391106, -0.8982084 , -0.9997953 , -0.34002233, -0.40260598,
       -0.31840608, -0.87243557, -0.85644567, -0.47168043, -0.9281716 ,
       -1.        , -0.08342128, -0.1261841 , -0.99650866], dtype=float32), input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='MaximumTokenProbability')


In [3]:
estimator = MeanTokenEntropy()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=1.3938677310943604, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='MeanTokenEntropy')

In [4]:
estimator = MeanPointwiseMutualInformation()
estimate_uncertainty(model, estimator, input_text=input_text)

Keyword argument `return_dict` is not a valid argument for this processor and will be ignored.


UncertaintyOutput(uncertainty=-10.939052400236687, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='MeanPointwiseMutualInformation')

In [5]:
estimator = MeanConditionalPointwiseMutualInformation()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-30.574629389362894, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='MeanConditionalPointwiseMutualInformation')

In [6]:
estimator = ClaimConditionedProbability()
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=-0.0024776210738792826, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='CCP')

In [7]:
estimator = MonteCarloNormalizedSequenceEntropy()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=1.7204913228075853, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='MonteCarloNormalizedSequenceEntropy')

In [8]:
estimator = MonteCarloSequenceEntropy()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=47.520239404774586, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='MonteCarloSequenceEntropy')

In [9]:
estimator = LexicalSimilarity(metric="rouge1")
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-0.4897957552999105, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='LexicalSimilarity_rouge1')

In [10]:
estimator = LexicalSimilarity(metric="rouge2")
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-0.23516591475625653, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='LexicalSimilarity_rouge2')

In [11]:
estimator = LexicalSimilarity(metric="rougeL")
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-0.3957066452121844, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='LexicalSimilarity_rougeL')

In [ ]:
estimator = LexicalSimilarity(metric="BLEU")
estimate_uncertainty(model, estimator, input_text=input_text)

/home/jovyan/.mlspace/envs/focus_new/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jovyan/.mlspace/envs/focus_new/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jovyan/.mlspace/envs/focus_new/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.


UncertaintyOutput(uncertainty=-7.572732360373077e-80, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='LexicalSimilarity_BLEU')

In [ ]:
estimator = NumSemSets()
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=1.0, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='NumSemSets')

In [ ]:
estimator = EigValLaplacian(similarity_score="NLI_score", affinity="entail")
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=5.096669971942901, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='EigValLaplacian_NLI_score_entail')

In [ ]:
estimator = EigValLaplacian(similarity_score="NLI_score", affinity="contra")
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=1.4312051618146395, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='EigValLaplacian_NLI_score_contra')

In [ ]:
estimator = EigValLaplacian(similarity_score="Jaccard_score")
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=5.915210228312991, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='EigValLaplacian_Jaccard_score')

In [ ]:
estimator = DegMat(similarity_score="NLI_score", affinity="entail")
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=0.7471796417236328, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='DegMat_NLI_score_entail')

In [ ]:
estimator = Eccentricity(similarity_score="NLI_score", affinity="entail")
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=2.828599438215806, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='Eccentricity_NLI_score_entail')

In [ ]:
estimator = Eccentricity(similarity_score="NLI_score", affinity="contra")
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=1.000979058779359, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='Eccentricity_NLI_score_contra')

In [ ]:
estimator = Eccentricity(similarity_score="Jaccard_score")
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=3.0, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='Eccentricity_Jaccard_score')

In [ ]:
estimator = SemanticEntropy()
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=35.55207718369826, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='SemanticEntropy')

In [ ]:
estimator = SAR()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-6.398743422786111, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='SAR')

In [ ]:
estimator = TokenSAR()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=0.6193434596061707, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='TokenSAR')

In [ ]:
estimator = SentenceSAR()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=9.301745888814732, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='SentenceSAR')

In [ ]:
estimator = RenyiNeg()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-20.160396575927734, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='RenyiNeg')

In [ ]:
estimator = FisherRao()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=0.6960842609405518, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='FisherRao')

In [ ]:
estimator = KernelLanguageEntropy()
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/jovyan/rvanova/lm-polygraph/src/lm_polygraph/estimators/kernel_language_entropy.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
  eigvs = np.linalg.eig(K + jitter * np.eye(K.shape[0])).eigenvalues.astype(


UncertaintyOutput(uncertainty=0.9251821835991644, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='KernelLanguageEntropy')

In [ ]:
estimator = LUQ()
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=0.5553479790687561, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='LUQ')

In [ ]:
estimator = EigenScore()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=13.372737963072636, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='EigenScore')

In [ ]:
model_name = model.model.config._name_or_path
estimator = Focus(
        model_name=model_name,
        path=f"../focus_data/{model_name}/token_idf.pkl",
        gamma=0.9,
        p=0.01,
        idf_dataset="togethercomputer/RedPajama-Data-1T-Sample",
        trust_remote_code=True,
        idf_seed=42,
        idf_dataset_size=1000,
        spacy_path="en_core_web_sm",
    )
estimate_uncertainty(model, estimator, input_text=input_text)

100%|██████████| 1000/1000 [00:04<00:00, 230.29it/s]


UncertaintyOutput(uncertainty=8.798990133744413, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='Focus (gamma=0.9)')

In [ ]:
estimator = PTrue()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=12.144603729248047, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='PTrue')

In [ ]:
estimator = PTrueSampling()
estimate_uncertainty(model, estimator, input_text=input_text)

In [ ]:
estimator = Perplexity()
estimate_uncertainty(model, estimator, input_text=input_text)

In [ ]:
estimator = MaximumSequenceProbability()
estimate_uncertainty(model, estimator, input_text=input_text)

In [ ]:
estimator = LexicalSimilarity('rougeL')
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=-0.4147621507026083, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='LexicalSimilarity_rougeL')

In [ ]:
estimator = SemanticEntropy()
estimate_uncertainty(model, estimator, input_text=input_text)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UncertaintyOutput(uncertainty=38.022790048390405, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='SemanticEntropy')

In [ ]:
estimator = PointwiseMutualInformation()
estimate_uncertainty(model, estimator, input_text=input_text)

UncertaintyOutput(uncertainty=0.8221019506454468, input_text=['<grounding>An image of'], generation_text='<phrase> A snowman</phrase><object><patch_index_0044><patch_index_0863></object> is sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in', generation_tokens=[64007, 95, 43867, 64008, 64009, 64057, 64876, 64010, 17, 1280, 32, 64007, 10, 30879, 64008, 64009, 64018, 65020, 64010], model_path=None, estimator='PointwiseMutualInformation')